In [4]:
import pyforest
import sys
sys.path.append('/Users/jayceepang/msse/ATOM_CODE/UTILS/')
from RF_functions import *
from GenUtils import *
from dataset import *
import os
import pickle
from imblearn.over_sampling import SMOTEN, ADASYN, SMOTE
from sklearn.metrics import accuracy_score, balanced_accuracy_score,precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix,matthews_corrcoef

from rdkit import Chem
from rdkit.Chem import AllChem

In [5]:
og_neks = ['NEK2_binding','NEK2_inhibition','NEK3_binding','NEK5_binding','NEK9_binding','NEK9_inhibition']
samplings =['none_scaled','UNDER','SMOTE','ADASYN'] 
new_neks=['NEK1', 'NEK11', 'NEK2', 'NEK3', 'NEK4', 'NEK5',
       'NEK6', 'NEK9']
RF_types = ['RF','RF_BCW','BRFC','BRFC_BCW']
og_datapath = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/'
results_dir='/Users/jayceepang/msse/ATOM_CODE/examples/holdout/holdout_results/'
datapath = '/Users/jayceepang/msse/ATOM_CODE/datasets/holdout_data/featurized/'
rf_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/RF_results/'
gp_path ='/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/GP_results/'
test_results=[]
final_cols=['model','NEK','strategy','feat_type','RF_type', 'cm','recall', 'specificity', 'accuracy', 'precision', 
                'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy']
feat_cols_dropped =['NEK','compound_id','base_rdkit_smiles', 'subset', 'active'] 

In [ ]:
test_results=[]
final_cols=['model','NEK','strategy','feat_type','RF_type', 'cm','recall', 'specificity', 'accuracy', 'precision', 
                'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy']
for nek in new_neks: 
    for i in og_neks: 
        for samp in samplings: 
            trainX=pd.read_csv(f'{og_datapath}{i}_MFP_{samp}_trainX.csv')
            train_y =pd.read_csv(f'{og_datapath}{i}_MFP_{samp}_train_y.csv').to_numpy().reshape(-1)
            testX=pd.read_csv(f'{datapath}{nek}_MFP_none_unscaled.csv').drop(columns=feat_cols_dropped)
            test_y=pd.read_csv(f'{datapath}{nek}_MFP_none_unscaled.csv')['active'].to_numpy().reshape(-1)
            for rf in RF_types: 
                model_name = f'{i}_MFP_{samp}_{rf}'
                
                print(nek, model_name) 
                model = RF_functions.rf_models(trainX, train_y, testX, test_y, rf, {}, True)
                # train_df = gather_rf_results(model, trainX, train_y)
                test_df = gather_rf_results(model, testX, test_y)
                print()

                test_df['model'] = f'{nek}_{model_name}'
                test_df['NEK'] =nek
                test_df['feat_type'] = 'MFP'
                test_df['strategy'] = samp
                test_df['RF_type'] = rf
                test_df.to_csv(f'{results_dir}{model_name}_holdout_test.csv',index=False) 
                test_results.append(test_df.iloc[[0]][final_cols].values.flatten())

    all_test =  pd.DataFrame(test_results,columns=final_cols)
    all_test['modeling_type'] = 'RF' 
    all_test['set'] = 'nanobret_cellular_holdout' 

    all_test.to_csv(f'{results_dir}RF_holdout_test_results_all_NEK.csv', index=False)                 
                        
        
